In [ ]:
import eventlet
import json
from eventlet import wsgi, websocket
import argparse
import logging

log = logging.getLogger('werkzeug')
log.setLevel(logging.ERROR)

In [ ]:
@websocket.WebSocketWSGI
def handle(ws):
    while True:
        message = ws.wait()
        if message is None: break
        data = json.loads(message)
        ws.send(json.dumps({ 'hello': data['name'] }))
            

In [ ]:
def site(env, start_response):
    if env['PATH_INFO'] == '/hello':
        return handle(env, start_response)
    else:
        start_response('200 OK', [('Content-Type', 'text/plain')])
        return ['Hello, Eventlet!']

In [ ]:
host = '0.0.0.0'
port = 5004
print('Server starting at: ' + 'ws://{}:{}'.format(host, port))
listener = eventlet.listen((host, port))
wsgi.server(listener, site)

In [1]:
from fastapi import FastAPI, WebSocket
import argparse
import uvicorn

In [2]:
app = FastAPI()

In [3]:
@app.websocket("/hello")
async def hello(websocket: WebSocket):
    await websocket.accept()
    while True:
        try:
            data = await websocket.receive_json()
            await websocket.send_json({ 'hello': data['name'] })
        except:
            await websocket.close()
            break

In [4]:
uvicorn.run(app, host='0.0.0.0', port=5004, log_level='error')

RuntimeError: This event loop is already running